In [1]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py
# https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
import os
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import multiprocessing
cores = multiprocessing.cpu_count()
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.test.utils import get_tmpfile
from sklearn.metrics import accuracy_score, f1_score
import collections


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nsuse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nsuse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-05-01 13:22:27,759 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2023-05-01 13:22:27,761 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2023-05-01 13:22:27,762 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2023-05-01T13:22:27.762886', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [3]:
cores = multiprocessing.cpu_count()
cores

20

In [4]:
completed_non_ea_bibs = []
completed_non_ea_text = []
with open('.//data//non_ea_papers_text.txt', 'r', encoding="utf-8") as f:
    for line in f:
        bib = line.split(",")[0].strip()
        text = line.split(",")[1].strip()
        completed_non_ea_bibs.append(bib)
        completed_non_ea_text.append(text)
n_neg = len(completed_non_ea_bibs)

In [5]:
completed_ea_bibs = []
completed_ea_text = []
with open('.//data//ea_papers_text.txt', 'r', encoding="utf-8") as f:
    for line in f:
        bib = line.split(",")[0].strip()
        text = line.split(",")[1].strip()
        completed_ea_bibs.append(bib)
        completed_ea_text.append(text)
n_pos = len(completed_ea_bibs)

In [6]:
pos_training_corpus = pd.DataFrame({"text":completed_ea_text, "bib":completed_ea_bibs, "label":np.repeat(1, len(completed_ea_bibs))})
neg_training_corpus = pd.DataFrame({"text":completed_non_ea_text, "bib":completed_non_ea_bibs, "label":np.repeat(0, len(completed_non_ea_bibs))})

training_corpus = pd.concat([pos_training_corpus, neg_training_corpus])
training_corpus = training_corpus.sample(frac=1, random_state=42).reset_index(drop=True)
training_corpus["index"] = training_corpus.index
training_corpus

,text,bib,label,index
0,arxivastroph0403324v4 17 jul 2022dark matter ...,2004LNP...653..141S,0,0
1,mon not r astron soc 000 19 0000 printed 20 au...,2013MNRAS.428.1077S,1,1
2,mon not r astron soc 427 3435–3467 2012 doi101...,2012MNRAS.427.3435A,0,2
3,astronomy astrophysics manuscript no hd20v2 ce...,2019arXiv190808754S,1,3
4,the microphysics of collisionless shock wavesa...,2016RPPh...79d6901M,0,4
...,...,...,...,...
7558,mon not r astron soc 360 869–891 2005 doi10111...,2005MNRAS.360..869L,0,7558
7559,arxivastroph0508228v1 10 aug 2005adarkjetdomi...,2005Natur.436..819G,0,7559
7560,1accepted for publication in nature20 october ...,2006Natur.444.1044G,0,7560
7561,arxiv11045230v3 astrophep 7 jul 2011accepted...,2011ApJ...737L..18W,1,7561


In [9]:
training_corpus["bib"].to_csv(".//data//training_bibs.csv", index=False)

In [25]:
print(n_neg)
print(n_pos)
n_neg/n_pos

4025
3538


1.137648388920294

In [27]:
n_pos/n_neg

0.8790062111801242

In [6]:
# neg_training_corpus_subset = neg_training_corpus.sample(n=n_pos, random_state=42).reset_index(drop=True)

# balanced_training_corpus = pd.concat([pos_training_corpus, neg_training_corpus_subset])
# balanced_training_corpus = balanced_training_corpus.sample(frac=1, random_state=42).reset_index(drop=True)

# len(balanced_training_corpus[balanced_training_corpus["label"] == 0].index) == len(balanced_training_corpus[balanced_training_corpus["label"] == 1].index)

In [7]:
stopWords = set(stopwords.words('english'))

def tokenize_text(text):
    tokens = []
    for sentence in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sentence):
            if len(word) < 2 or word in stopWords:
                continue
            tokens.append(word.lower())
    return tokens

# punctuation was removed and text was set to lower case earlier (Convert PDFs to Text.ipynb)

In [8]:
training_corpus_tagged = training_corpus.apply(lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r["index"]]), axis=1).tolist()

In [10]:
train_subset_tagged, test_subset_tagged = train_test_split(training_corpus_tagged, test_size=0.1, random_state=42)

In [11]:
# https://stackoverflow.com/questions/69762635/what-are-the-negative-sample-parameters

model = Doc2Vec(dm=0, vector_size=600, negative=5, sample=0, workers=cores, epochs=30)

model.build_vocab(train_subset_tagged)
model.train(train_subset_tagged, total_examples=model.corpus_count, epochs=model.epochs)

2023-02-23 11:02:30,597 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d600,n5,mc5,t20)', 'datetime': '2023-02-23T11:02:30.597754', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2023-02-23 11:02:30,601 : INFO : collecting all words and their counts
2023-02-23 11:02:30,601 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\nsuse\anaconda3\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Users\nsuse\anaconda3\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Users\nsuse\anaconda3\lib\logging\__init__.py", line 663, in format
    record.message = record.getMessage()
  File "C:\Users\nsuse\anaconda3\lib\logging\__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not enough

2023-02-23 11:03:14,820 : INFO : EPOCH 1 - PROGRESS: at 47.94% examples, 1211556 words/s, in_qsize 40, out_qsize 0
2023-02-23 11:03:15,845 : INFO : EPOCH 1 - PROGRESS: at 50.63% examples, 1199717 words/s, in_qsize 40, out_qsize 29
2023-02-23 11:03:16,926 : INFO : EPOCH 1 - PROGRESS: at 54.38% examples, 1212519 words/s, in_qsize 40, out_qsize 19
2023-02-23 11:03:17,879 : INFO : EPOCH 1 - PROGRESS: at 57.67% examples, 1213822 words/s, in_qsize 40, out_qsize 6
2023-02-23 11:03:18,884 : INFO : EPOCH 1 - PROGRESS: at 61.03% examples, 1215330 words/s, in_qsize 37, out_qsize 4
2023-02-23 11:03:19,904 : INFO : EPOCH 1 - PROGRESS: at 64.18% examples, 1216128 words/s, in_qsize 40, out_qsize 7
2023-02-23 11:03:20,908 : INFO : EPOCH 1 - PROGRESS: at 67.57% examples, 1218583 words/s, in_qsize 39, out_qsize 0
2023-02-23 11:03:21,910 : INFO : EPOCH 1 - PROGRESS: at 70.63% examples, 1215518 words/s, in_qsize 40, out_qsize 9
2023-02-23 11:03:22,927 : INFO : EPOCH 1 - PROGRESS: at 74.11% examples, 12199

2023-02-23 11:04:01,820 : INFO : worker thread finished; awaiting finish of 8 more threads
2023-02-23 11:04:01,820 : INFO : worker thread finished; awaiting finish of 7 more threads
2023-02-23 11:04:01,824 : INFO : worker thread finished; awaiting finish of 6 more threads
2023-02-23 11:04:01,824 : INFO : worker thread finished; awaiting finish of 5 more threads
2023-02-23 11:04:01,824 : INFO : worker thread finished; awaiting finish of 4 more threads
2023-02-23 11:04:01,824 : INFO : worker thread finished; awaiting finish of 3 more threads
2023-02-23 11:04:01,840 : INFO : worker thread finished; awaiting finish of 2 more threads
2023-02-23 11:04:01,844 : INFO : worker thread finished; awaiting finish of 1 more threads
2023-02-23 11:04:01,848 : INFO : worker thread finished; awaiting finish of 0 more threads
2023-02-23 11:04:01,852 : INFO : EPOCH - 2 : training on 47823808 raw words (37665975 effective words) took 31.2s, 1208843 effective words/s
2023-02-23 11:04:02,887 : INFO : EPOCH 3

2023-02-23 11:04:50,314 : INFO : EPOCH 4 - PROGRESS: at 50.26% examples, 1151845 words/s, in_qsize 39, out_qsize 0
2023-02-23 11:04:51,319 : INFO : EPOCH 4 - PROGRESS: at 53.45% examples, 1153708 words/s, in_qsize 39, out_qsize 0
2023-02-23 11:04:52,325 : INFO : EPOCH 4 - PROGRESS: at 56.46% examples, 1153862 words/s, in_qsize 12, out_qsize 0
2023-02-23 11:04:53,334 : INFO : EPOCH 4 - PROGRESS: at 59.64% examples, 1154913 words/s, in_qsize 38, out_qsize 1
2023-02-23 11:04:54,381 : INFO : EPOCH 4 - PROGRESS: at 62.52% examples, 1152546 words/s, in_qsize 36, out_qsize 3
2023-02-23 11:04:55,434 : INFO : EPOCH 4 - PROGRESS: at 65.72% examples, 1152405 words/s, in_qsize 40, out_qsize 11
2023-02-23 11:04:56,451 : INFO : EPOCH 4 - PROGRESS: at 68.95% examples, 1154459 words/s, in_qsize 40, out_qsize 3
2023-02-23 11:04:57,451 : INFO : EPOCH 4 - PROGRESS: at 72.04% examples, 1155478 words/s, in_qsize 37, out_qsize 0
2023-02-23 11:04:58,469 : INFO : EPOCH 4 - PROGRESS: at 75.10% examples, 115495

2023-02-23 11:05:38,322 : INFO : worker thread finished; awaiting finish of 10 more threads
2023-02-23 11:05:38,323 : INFO : worker thread finished; awaiting finish of 9 more threads
2023-02-23 11:05:38,324 : INFO : worker thread finished; awaiting finish of 8 more threads
2023-02-23 11:05:38,324 : INFO : worker thread finished; awaiting finish of 7 more threads
2023-02-23 11:05:38,326 : INFO : worker thread finished; awaiting finish of 6 more threads
2023-02-23 11:05:38,327 : INFO : worker thread finished; awaiting finish of 5 more threads
2023-02-23 11:05:38,329 : INFO : worker thread finished; awaiting finish of 4 more threads
2023-02-23 11:05:38,330 : INFO : worker thread finished; awaiting finish of 3 more threads
2023-02-23 11:05:38,334 : INFO : worker thread finished; awaiting finish of 2 more threads
2023-02-23 11:05:38,335 : INFO : worker thread finished; awaiting finish of 1 more threads
2023-02-23 11:05:38,338 : INFO : worker thread finished; awaiting finish of 0 more thread

...

2023-02-23 11:15:03,229 : INFO : worker thread finished; awaiting finish of 19 more threads
2023-02-23 11:15:03,241 : INFO : worker thread finished; awaiting finish of 18 more threads
2023-02-23 11:15:03,241 : INFO : worker thread finished; awaiting finish of 17 more threads
2023-02-23 11:15:03,245 : INFO : worker thread finished; awaiting finish of 16 more threads
2023-02-23 11:15:03,245 : INFO : worker thread finished; awaiting finish of 15 more threads
2023-02-23 11:15:03,245 : INFO : worker thread finished; awaiting finish of 14 more threads
2023-02-23 11:15:03,249 : INFO : worker thread finished; awaiting finish of 13 more threads
2023-02-23 11:15:03,253 : INFO : worker thread finished; awaiting finish of 12 more threads
2023-02-23 11:15:03,259 : INFO : worker thread finished; awaiting finish of 11 more threads
2023-02-23 11:15:03,259 : INFO : worker thread finished; awaiting finish of 10 more threads
2023-02-23 11:15:03,264 : INFO : worker thread finished; awaiting finish of 9 mo

2023-02-23 11:15:42,336 : INFO : EPOCH 26 - PROGRESS: at 36.19% examples, 1316375 words/s, in_qsize 40, out_qsize 5
2023-02-23 11:15:43,340 : INFO : EPOCH 26 - PROGRESS: at 39.82% examples, 1316119 words/s, in_qsize 40, out_qsize 5
2023-02-23 11:15:44,373 : INFO : EPOCH 26 - PROGRESS: at 43.48% examples, 1313734 words/s, in_qsize 40, out_qsize 10
2023-02-23 11:15:45,437 : INFO : EPOCH 26 - PROGRESS: at 46.80% examples, 1306432 words/s, in_qsize 40, out_qsize 28
2023-02-23 11:15:46,439 : INFO : EPOCH 26 - PROGRESS: at 50.71% examples, 1318014 words/s, in_qsize 39, out_qsize 0
2023-02-23 11:15:47,449 : INFO : EPOCH 26 - PROGRESS: at 54.20% examples, 1318962 words/s, in_qsize 40, out_qsize 1
2023-02-23 11:15:48,454 : INFO : EPOCH 26 - PROGRESS: at 57.60% examples, 1317164 words/s, in_qsize 40, out_qsize 6
2023-02-23 11:15:49,535 : INFO : EPOCH 26 - PROGRESS: at 60.67% examples, 1303994 words/s, in_qsize 40, out_qsize 48
2023-02-23 11:15:50,565 : INFO : EPOCH 26 - PROGRESS: at 64.55% examp

2023-02-23 11:16:31,247 : INFO : worker thread finished; awaiting finish of 10 more threads
2023-02-23 11:16:31,248 : INFO : worker thread finished; awaiting finish of 9 more threads
2023-02-23 11:16:31,249 : INFO : worker thread finished; awaiting finish of 8 more threads
2023-02-23 11:16:31,250 : INFO : worker thread finished; awaiting finish of 7 more threads
2023-02-23 11:16:31,251 : INFO : worker thread finished; awaiting finish of 6 more threads
2023-02-23 11:16:31,252 : INFO : worker thread finished; awaiting finish of 5 more threads
2023-02-23 11:16:31,254 : INFO : worker thread finished; awaiting finish of 4 more threads
2023-02-23 11:16:31,257 : INFO : worker thread finished; awaiting finish of 3 more threads
2023-02-23 11:16:31,260 : INFO : worker thread finished; awaiting finish of 2 more threads
2023-02-23 11:16:31,261 : INFO : worker thread finished; awaiting finish of 1 more threads
2023-02-23 11:16:31,269 : INFO : worker thread finished; awaiting finish of 0 more thread

2023-02-23 11:17:18,759 : INFO : EPOCH 29 - PROGRESS: at 63.08% examples, 1274747 words/s, in_qsize 40, out_qsize 4
2023-02-23 11:17:19,805 : INFO : EPOCH 29 - PROGRESS: at 66.53% examples, 1274189 words/s, in_qsize 20, out_qsize 19
2023-02-23 11:17:20,845 : INFO : EPOCH 29 - PROGRESS: at 70.47% examples, 1281156 words/s, in_qsize 30, out_qsize 9
2023-02-23 11:17:21,905 : INFO : EPOCH 29 - PROGRESS: at 73.88% examples, 1280049 words/s, in_qsize 40, out_qsize 17
2023-02-23 11:17:22,927 : INFO : EPOCH 29 - PROGRESS: at 77.62% examples, 1285410 words/s, in_qsize 25, out_qsize 7
2023-02-23 11:17:23,929 : INFO : EPOCH 29 - PROGRESS: at 81.38% examples, 1290124 words/s, in_qsize 38, out_qsize 3
2023-02-23 11:17:24,934 : INFO : EPOCH 29 - PROGRESS: at 84.98% examples, 1290781 words/s, in_qsize 40, out_qsize 2
2023-02-23 11:17:25,970 : INFO : EPOCH 29 - PROGRESS: at 88.58% examples, 1291327 words/s, in_qsize 40, out_qsize 6
2023-02-23 11:17:26,986 : INFO : EPOCH 29 - PROGRESS: at 91.87% exampl

2023-02-23 11:17:57,399 : INFO : EPOCH - 30 : training on 47823808 raw words (37665975 effective words) took 28.3s, 1332628 effective words/s
2023-02-23 11:17:57,399 : INFO : Doc2Vec lifecycle event {'msg': 'training on 1434714240 raw words (1129979250 effective words) took 898.0s, 1258336 effective words/s', 'datetime': '2023-02-23T11:17:57.399018', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'train'}


In [12]:
len(model.wv)

339129

In [13]:
len(model.dv)

7563

In [15]:
ranks = []
for doc_id in range(len(train_subset_tagged)):
    inferred_vector = model.infer_vector(train_subset_tagged[doc_id].words)
    tag = train_subset_tagged[doc_id].tags[0]
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(tag)
    ranks.append(rank)

counter = collections.Counter(ranks)
print(counter)
ranking_accuracy = counter[0]/(sum([counter[i] for i in range(len(counter))]))
print(ranking_accuracy)

Counter({0: 6753, 1: 53})
0.9922127534528358


In [16]:
correct = 0
for doc_id in range(len(test_subset_tagged)):
    inferred_vector = model.infer_vector(test_subset_tagged[doc_id].words)
    tag = test_subset_tagged[doc_id].tags[0]
    true_label = training_corpus[training_corpus["index"]==tag]["label"].values[0]
    
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    scores0 = []
    scores1 = []
    for docid, weight in sims:
        label = training_corpus[training_corpus["index"]==docid]["label"].values[0]
        if label == 0:
            scores0.append(weight)
        elif label == 1:
            scores1.append(weight)
            
    predicted_label = np.argmax([np.mean(scores0), np.mean(scores1)])
    correct += (predicted_label == true_label)
    
print(correct/len(test_subset_tagged))

0.964332892998679


In [17]:
# doc2vec model assessments
# dm \\ vector_size \\ negative \\ sample \\ train accuracy     \\ test accuracy
# 0  \\ 300         \\ 5        \\ 0      \\ 0.9922127534528358 \\ 0.9603698811096433
# 1  \\ 300         \\ 5        \\ 0      \\ 0.9908903908316191 \\ 0.9577278731836195
# 0  \\ 600         \\ 5        \\ 0      \\ 0.9925066118131061  \\ 0.964332892998679
# 1  \\ 600         \\ 5        \\ 0      \\ 0.9904496032912137  \\ 0.9590488771466315
# 0  \\ 900         \\ 5        \\ 0      \\ 0.9922127534528358 \\ 0.9616908850726552
# 0  \\ 1200        \\ 5        \\ 0      \\ 0.9920658242727005 \\ 0.9616908850726552
# 1  \\ 1200        \\ 5        \\ 0      \\ 0.9913311783720247 \\ 0.9603698811096433
# 0  \\ 600         \\ 10       \\ 0      \\ 0.9920658242727005  \\ 0.9630118890356671
# 0  \\ 600         \\ 20       \\ 0      \\ 0.9920658242727005  \\ 0.9630118890356671
# 0  \\ 600         \\ 5        \\ 1e-5   \\ 0.9920658242727005  \\ 0.9630118890356671


In [19]:
model = Doc2Vec(dm=0, vector_size=600, negative=5, sample=0, workers=cores, epochs=30)

# train final model on all data
model.build_vocab(training_corpus_tagged)
model.train(training_corpus_tagged, total_examples=model.corpus_count, epochs=model.epochs)

2023-02-23 14:01:58,383 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d600,n5,mc5,t20)', 'datetime': '2023-02-23T14:01:58.383137', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2023-02-23 14:02:00,028 : INFO : collecting all words and their counts
2023-02-23 14:02:00,028 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2023-02-23 14:02:27,650 : INFO : collected 5592890 word types and 7563 unique tags from a corpus of 7563 examples and 53124816 words
2023-02-23 14:02:27,650 : INFO : Creating a fresh vocabulary
2023-02-23 14:02:31,629 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 369198 unique words (6.6012025982989115%% of original 5592890, drops 5223692)', 'datetime': '2023-02-23T14:02:31.629467', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Wi

2023-02-23 14:03:18,347 : INFO : EPOCH - 1 : training on 53124816 raw words (41917369 effective words) took 35.0s, 1197046 effective words/s
2023-02-23 14:03:19,365 : INFO : EPOCH 2 - PROGRESS: at 2.64% examples, 1073781 words/s, in_qsize 30, out_qsize 5
2023-02-23 14:03:20,370 : INFO : EPOCH 2 - PROGRESS: at 5.71% examples, 1177800 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:03:21,382 : INFO : EPOCH 2 - PROGRESS: at 8.49% examples, 1171887 words/s, in_qsize 37, out_qsize 3
2023-02-23 14:03:22,384 : INFO : EPOCH 2 - PROGRESS: at 11.44% examples, 1176144 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:03:23,388 : INFO : EPOCH 2 - PROGRESS: at 14.31% examples, 1171967 words/s, in_qsize 26, out_qsize 4
2023-02-23 14:03:24,390 : INFO : EPOCH 2 - PROGRESS: at 17.29% examples, 1181749 words/s, in_qsize 29, out_qsize 0
2023-02-23 14:03:25,392 : INFO : EPOCH 2 - PROGRESS: at 20.14% examples, 1181025 words/s, in_qsize 34, out_qsize 0
2023-02-23 14:03:26,394 : INFO : EPOCH 2 - PROGRESS: at 2

2023-02-23 14:04:13,323 : INFO : EPOCH 3 - PROGRESS: at 57.05% examples, 1181211 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:04:14,389 : INFO : EPOCH 3 - PROGRESS: at 60.00% examples, 1180130 words/s, in_qsize 32, out_qsize 7
2023-02-23 14:04:15,389 : INFO : EPOCH 3 - PROGRESS: at 63.00% examples, 1183299 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:04:16,388 : INFO : EPOCH 3 - PROGRESS: at 65.77% examples, 1182634 words/s, in_qsize 33, out_qsize 1
2023-02-23 14:04:17,399 : INFO : EPOCH 3 - PROGRESS: at 68.56% examples, 1183978 words/s, in_qsize 28, out_qsize 2
2023-02-23 14:04:18,402 : INFO : EPOCH 3 - PROGRESS: at 71.10% examples, 1180592 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:04:19,398 : INFO : EPOCH 3 - PROGRESS: at 73.87% examples, 1178704 words/s, in_qsize 40, out_qsize 0
2023-02-23 14:04:20,399 : INFO : EPOCH 3 - PROGRESS: at 76.61% examples, 1177433 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:04:21,399 : INFO : EPOCH 3 - PROGRESS: at 79.68% examples, 1178440

2023-02-23 14:05:03,803 : INFO : worker thread finished; awaiting finish of 13 more threads
2023-02-23 14:05:03,807 : INFO : worker thread finished; awaiting finish of 12 more threads
2023-02-23 14:05:03,811 : INFO : worker thread finished; awaiting finish of 11 more threads
2023-02-23 14:05:03,818 : INFO : worker thread finished; awaiting finish of 10 more threads
2023-02-23 14:05:03,820 : INFO : worker thread finished; awaiting finish of 9 more threads
2023-02-23 14:05:03,820 : INFO : worker thread finished; awaiting finish of 8 more threads
2023-02-23 14:05:03,820 : INFO : worker thread finished; awaiting finish of 7 more threads
2023-02-23 14:05:03,832 : INFO : worker thread finished; awaiting finish of 6 more threads
2023-02-23 14:05:03,838 : INFO : worker thread finished; awaiting finish of 5 more threads
2023-02-23 14:05:03,844 : INFO : worker thread finished; awaiting finish of 4 more threads
2023-02-23 14:05:03,846 : INFO : worker thread finished; awaiting finish of 3 more thr

2023-02-23 14:05:48,088 : INFO : EPOCH 6 - PROGRESS: at 26.62% examples, 1209070 words/s, in_qsize 38, out_qsize 5
2023-02-23 14:05:49,093 : INFO : EPOCH 6 - PROGRESS: at 29.56% examples, 1213813 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:05:50,099 : INFO : EPOCH 6 - PROGRESS: at 32.53% examples, 1214889 words/s, in_qsize 38, out_qsize 1
2023-02-23 14:05:51,103 : INFO : EPOCH 6 - PROGRESS: at 35.58% examples, 1217473 words/s, in_qsize 40, out_qsize 0
2023-02-23 14:05:52,111 : INFO : EPOCH 6 - PROGRESS: at 38.50% examples, 1217059 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:05:53,128 : INFO : EPOCH 6 - PROGRESS: at 41.25% examples, 1210501 words/s, in_qsize 40, out_qsize 7
2023-02-23 14:05:54,134 : INFO : EPOCH 6 - PROGRESS: at 44.20% examples, 1212600 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:05:55,145 : INFO : EPOCH 6 - PROGRESS: at 47.23% examples, 1216475 words/s, in_qsize 32, out_qsize 0
2023-02-23 14:05:56,157 : INFO : EPOCH 6 - PROGRESS: at 50.06% examples, 1213261

...

2023-02-23 14:16:11,255 : INFO : worker thread finished; awaiting finish of 8 more threads
2023-02-23 14:16:11,256 : INFO : worker thread finished; awaiting finish of 7 more threads
2023-02-23 14:16:11,258 : INFO : worker thread finished; awaiting finish of 6 more threads
2023-02-23 14:16:11,263 : INFO : worker thread finished; awaiting finish of 5 more threads
2023-02-23 14:16:11,265 : INFO : worker thread finished; awaiting finish of 4 more threads
2023-02-23 14:16:11,269 : INFO : worker thread finished; awaiting finish of 3 more threads
2023-02-23 14:16:11,274 : INFO : worker thread finished; awaiting finish of 2 more threads
2023-02-23 14:16:11,276 : INFO : worker thread finished; awaiting finish of 1 more threads
2023-02-23 14:16:11,277 : INFO : worker thread finished; awaiting finish of 0 more threads
2023-02-23 14:16:11,278 : INFO : EPOCH - 24 : training on 53124816 raw words (41917369 effective words) took 32.8s, 1279679 effective words/s
2023-02-23 14:16:12,290 : INFO : EPOCH 

2023-02-23 14:16:59,531 : INFO : EPOCH 26 - PROGRESS: at 50.48% examples, 1308926 words/s, in_qsize 40, out_qsize 1
2023-02-23 14:17:00,555 : INFO : EPOCH 26 - PROGRESS: at 53.31% examples, 1300510 words/s, in_qsize 36, out_qsize 10
2023-02-23 14:17:01,556 : INFO : EPOCH 26 - PROGRESS: at 56.67% examples, 1305043 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:17:02,560 : INFO : EPOCH 26 - PROGRESS: at 59.74% examples, 1304077 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:17:03,567 : INFO : EPOCH 26 - PROGRESS: at 62.75% examples, 1301525 words/s, in_qsize 37, out_qsize 2
2023-02-23 14:17:04,569 : INFO : EPOCH 26 - PROGRESS: at 65.87% examples, 1303670 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:17:05,570 : INFO : EPOCH 26 - PROGRESS: at 68.99% examples, 1303728 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:17:06,590 : INFO : EPOCH 26 - PROGRESS: at 72.01% examples, 1302735 words/s, in_qsize 40, out_qsize 2
2023-02-23 14:17:07,609 : INFO : EPOCH 26 - PROGRESS: at 75.18% example

2023-02-23 14:17:47,586 : INFO : worker thread finished; awaiting finish of 10 more threads
2023-02-23 14:17:47,589 : INFO : worker thread finished; awaiting finish of 9 more threads
2023-02-23 14:17:47,594 : INFO : worker thread finished; awaiting finish of 8 more threads
2023-02-23 14:17:47,601 : INFO : worker thread finished; awaiting finish of 7 more threads
2023-02-23 14:17:47,603 : INFO : worker thread finished; awaiting finish of 6 more threads
2023-02-23 14:17:47,604 : INFO : worker thread finished; awaiting finish of 5 more threads
2023-02-23 14:17:47,606 : INFO : worker thread finished; awaiting finish of 4 more threads
2023-02-23 14:17:47,607 : INFO : worker thread finished; awaiting finish of 3 more threads
2023-02-23 14:17:47,609 : INFO : worker thread finished; awaiting finish of 2 more threads
2023-02-23 14:17:47,611 : INFO : worker thread finished; awaiting finish of 1 more threads
2023-02-23 14:17:47,619 : INFO : worker thread finished; awaiting finish of 0 more thread

2023-02-23 14:18:33,709 : INFO : EPOCH 29 - PROGRESS: at 45.10% examples, 1327150 words/s, in_qsize 40, out_qsize 0
2023-02-23 14:18:34,742 : INFO : EPOCH 29 - PROGRESS: at 48.27% examples, 1325223 words/s, in_qsize 36, out_qsize 3
2023-02-23 14:18:35,747 : INFO : EPOCH 29 - PROGRESS: at 51.45% examples, 1327950 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:18:36,761 : INFO : EPOCH 29 - PROGRESS: at 54.75% examples, 1326319 words/s, in_qsize 40, out_qsize 3
2023-02-23 14:18:37,761 : INFO : EPOCH 29 - PROGRESS: at 57.99% examples, 1330378 words/s, in_qsize 39, out_qsize 0
2023-02-23 14:18:38,803 : INFO : EPOCH 29 - PROGRESS: at 61.15% examples, 1325193 words/s, in_qsize 39, out_qsize 9
2023-02-23 14:18:39,819 : INFO : EPOCH 29 - PROGRESS: at 64.42% examples, 1326518 words/s, in_qsize 33, out_qsize 6
2023-02-23 14:18:40,828 : INFO : EPOCH 29 - PROGRESS: at 67.67% examples, 1329375 words/s, in_qsize 40, out_qsize 1
2023-02-23 14:18:41,840 : INFO : EPOCH 29 - PROGRESS: at 70.57% examples

2023-02-23 14:19:22,788 : INFO : worker thread finished; awaiting finish of 13 more threads
2023-02-23 14:19:22,792 : INFO : worker thread finished; awaiting finish of 12 more threads
2023-02-23 14:19:22,792 : INFO : worker thread finished; awaiting finish of 11 more threads
2023-02-23 14:19:22,796 : INFO : worker thread finished; awaiting finish of 10 more threads
2023-02-23 14:19:22,796 : INFO : worker thread finished; awaiting finish of 9 more threads
2023-02-23 14:19:22,800 : INFO : worker thread finished; awaiting finish of 8 more threads
2023-02-23 14:19:22,800 : INFO : worker thread finished; awaiting finish of 7 more threads
2023-02-23 14:19:22,804 : INFO : worker thread finished; awaiting finish of 6 more threads
2023-02-23 14:19:22,808 : INFO : worker thread finished; awaiting finish of 5 more threads
2023-02-23 14:19:22,812 : INFO : worker thread finished; awaiting finish of 4 more threads
2023-02-23 14:19:22,816 : INFO : worker thread finished; awaiting finish of 3 more thr

In [21]:
fname = ".//models//doc2vec//model_dbow_vs_600_n_5_s_0.bin"
model.save(fname)

2023-02-23 14:20:13,746 : INFO : Doc2Vec lifecycle event {'fname_or_handle': './/models//doc2vec//model_dbow_vs_600_n_5_s_0.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-02-23T14:20:13.746165', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'saving'}
2023-02-23 14:20:13,747 : INFO : storing np array 'vectors' to .//models//doc2vec//model_dbow_vs_600_n_5_s_0.bin.wv.vectors.npy
2023-02-23 14:20:14,713 : INFO : storing np array 'syn1neg' to .//models//doc2vec//model_dbow_vs_600_n_5_s_0.bin.syn1neg.npy
2023-02-23 14:20:15,819 : INFO : not storing attribute cum_table
2023-02-23 14:20:16,081 : INFO : saved .//models//doc2vec//model_dbow_vs_600_n_5_s_0.bin
